# Pelican PM Analysis III - Star Subset
*Author: Eric G. Suchanek, PhD, 3/21/19*

I noticed a population of stars within the fields of the Pelican Nebula PM analysis conducted by Bruce and Indigo. These stars had a PM direction angle $\theta$ in the 220-230$^\circ$ range. This peak is present in many if not all of the fields I surveyed in the PM batch run conducted earlier. As a result, I decided to grab this subpopulation to look at it a bit more closely. This code can read a .csv created by the GAIA query and 'slice' the stars into a specific angular range, and write that to a .csv file. The various graphs indicate that the slicing works properly. Interestingly, the subpopulation does not appear to be localized to one particular area of the field. I then did a linear regression of PMDec against PMRa using the scikit-learn regression model. This produced a fairly good correlation across the population extracted. Finally I used the PDF fitting algorithm to see if it will fit a PDF to the angular distribution histogram.

## Methodology

* Perform a query against the Gaia DR2 database, using a cone search over one of the star fields analyzed to date. These are typically 5-10 arcmin in radius.
* Extract the stars whose PM direction angle is within the range of interest
* Plot the PMRA and PMDec to look for any relationships




# Important Note

This version uses a different data-structure to manipulate the data returned from the Gaia query.  It uses pandas dataframes, not the list returned  by the Gaia query. To do this we run a normal Gaia search and return the job list result as normal, writing the results to a .csv file. The .csv file is then re-read into the program using pandas. As a result, all iterators were changed to reflect the new structure. _This means that one cannot necessarily cut and paste into and out of this code unless you're using the pandas data model._


In [1]:
#
# This version breaks from all prior code because it uses pandas dataframes, not the list returned
# by the Gaia query. This list is returned as normal, written to a .csv file, and then re-read into
# the program using pandas. As a result, all iterators were changed to reflect the new structure.

# job results from the Gaia query
import astropy.units as u
import astropy.coordinates as coord

from astropy.coordinates import Latitude, Longitude, Angle, SkyCoord
from astroquery.gaia import Gaia
from astropy.visualization import astropy_mpl_style
from astropy.io import ascii
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import progressbar

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

%matplotlib inline
import pandas as pd
from pandas import read_csv 
from pandas import DataFrame

# Suppress warnings. Comment this out if you wish to see the warning messages
import warnings
warnings.filterwarnings('ignore')

#from egs import compute_pm_angles

##
"""
def pm2ang(x, y):
    phi = np.arctan2(x,y)
    phi = phi * 180.0 / np.pi # to degrees
    phi = 90-phi
    return(phi % 360)


"""
# bruce's version
def pm2ang(x, y):
    phi = np.arctan(x/y)
    phi = phi * 180.0 / np.pi # to degrees
    if (y < 0):
        phi += 180
    return(phi % 360)

# gaia job list result
def compute_pm_angles(stars):    
    tot = len(stars)
    ang = [0.0] * tot
    for i in range(tot):
        ang[i] = bruce(stars[i]['pmra'],stars[i]['pmdec'])
    return(ang)

def compute_pm_angles_coords(pmra,pmdec):    
    tot = len(pmra)
    ang = []
    i = 0
    for i in range(tot):
         _pmra = pmra[i]
         _pmdec = pmdec[i]
         ang.append(pm2ang(_pmra,_pmdec))
    return(ang)

#
# the below routines work on pandas dataframes, not the list returned from the Gaia queries!
# it assumes that the gaia results have been saved and re-loaded from read_csv 
# compute proper motion angle, return in degrees, correct for quadrant

# return a list of theta angle for a pandas star list
def compute_pm_angles_pandas2(stars):
    tot = len(stars)
    ang = [0.0] * tot
    for index, row in stars.iterrows():
        pmra = row['pmra']
        pmdec = row['pmdec']
        ang[index] = pm2ang(pmra,pmdec)
    return(ang)

def compute_pm_angles_pandas(stars):
    tot = len(stars)
    pmra = stars['pmra']
    pmdec = stars['pmdec']
    ang = []
    for i in range(tot):
        _pmra = pmra[i]
        _pmdec = pmdec[i]
        ang.append(pm2ang(_pmra,_pmdec))
    return(ang)



#
# pandas dataframe iterator
# return a pandas subframe matching the low and high cutoffs
def slice_range_stars_pandas(stars, low, high):
    star_list = []
    cnt = 0
    i = 0
    for index, row in stars.iterrows():
        # print(row['c1'], row['c2'])
        pmra = row['pmra']
        pmdec = row['pmdec']
        if (pm2ang(pmra,pmdec) >= low) and (pm2ang(pmra,pmdec) <= high):
            star_list.append(row)
            cnt += 1
    return(star_list, cnt)

# returns pmra, pmdec, ra, dec lists which represent the stars
# whose pmra and pmdec and cnt for stars between the low and high cutoffs

def slice_range_coords_pandas(stars, low, high):
    pmra_list = []
    pmdec_list = []
    ra_list = []
    dec_list = []
    cnt = 0
    for index, row in stars.iterrows():
        pmra = row['pmra']
        pmdec = row['pmdec']
        ra = row['ra']
        dec = row['dec']
        ang = pm2ang(pmra,pmdec)
        if (ang >= low and ang <= high):
            pmra_list.append(pmra)
            pmdec_list.append(pmdec)
            ra_list.append(ra)
            dec_list.append(dec)
            cnt += 1
    return(pmra_list, pmdec_list, ra_list, dec_list, cnt)


# search parameters
_ra = "20h48m07s"
_dec = "36d26m17s"
_radius = 6
_area = 170

sel_str_circle = "SELECT * \
FROM gaiadr2.gaia_source \
WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',{},{},{}))=1 \
AND gaia_source.pmra IS NOT NULL \
AND gaia_source.pmdec IS NOT NULL \
AND gaia_source.parallax IS NOT NULL ;"

data_filename = "pm_out.csv"
center_coordinate = SkyCoord(ra=_ra, dec=_dec, 
                             unit=(u.hourangle, u.degree), 
                             frame='icrs')
radius = (_radius/60)*u.deg  # given in minutes, convert to degrees

query_str = sel_str_circle.format(center_coordinate.ra.deg, center_coordinate.dec.deg, 
                                  radius.value)
start = time.time()
job2 = Gaia.launch_job_async(query=query_str, dump_to_file=True, output_format="csv", 
                             output_file=data_filename)

stars_all = job2.get_results()
stars_pm = read_csv(data_filename)
end = time.time()
star_count = len(stars_pm)  # global variable

print("Elapsed time was %.4f Stars: %d" % (end - start, star_count))

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Elapsed time was 13.8197 Stars: 4512


In [2]:
pmra = stars_pm['pmra']
pmdec = stars_pm['pmdec']

ra = stars_pm['ra']
dec = stars_pm['dec']

n_bins = 36 # for the histogram - every 10 degrees

###########################################

# PM angle constraints for stars to extract
angle_start = 210.0
angle_end = 220

ra_narrow = []
dec_narrow = []
angle_narrow = []
pmra_nar = []
pmdec_nar = []
match_cnt = 0

# extract the pmra and pmdec lists of stars satisfying our angle contraints
pmra_nar, pmdec_nar, ra_narrow, dec_narrow, match_cnt = slice_range_coords_pandas(stars_pm, angle_start, angle_end)

# compute the angles for the extracted list
pm_angles = compute_pm_angles(pmra_nar, pmdec_nar)
pm_angles_all = compute_pm_angles(pmra,pmdec)

# write a file with the sliced stars only
slice_filename = "slice.csv"

sliced_stars = DataFrame()

sliced_stars['RA'] = ra_narrow
sliced_stars['Dec'] = dec_narrow
sliced_stars['PMRA'] = pmra_nar
sliced_stars['PMDec'] = pmdec_nar
sliced_stars['PM Theta'] = pm_angles

sliced_stars.to_csv(slice_filename)

####### Linear Regression ###########
pmra_narrow = np.asarray(pmra_nar)
pmdec_narrow = np.asarray(pmdec_nar)

subset_length = len(pmra_narrow)

# required for the scikit learn regressor model
pmra_reshaped = pmra_narrow.reshape(-1,1)
pmdec_reshaped = pmdec_narrow.reshape(-1,1)

# compute the linear regression with RA as X and Dec as Y
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(pmra_reshaped, pmdec_reshaped)  # perform linear regression

# now do the modeled prediction for Dec (Y in this case)
pmdec_pred = linear_regressor.predict(pmra_reshaped)  # make predictions

# Regression coefficients:
# mean standard error for the predicted against actual Y (pmdec)
_mse = mean_squared_error(pmdec_reshaped,pmdec_pred)

# Correlaton coefficient, R^2: 1 is perfect prediction
_corr_coeff = r2_score(pmdec_reshaped,pmdec_pred)

############# Plots ################

# 1. show a histogram of the angles for this query

title_string = str.format("PM Analysis for: %s to %s, %d/%d" % (_ra, _dec, subset_length, star_count))
position_string = str.format("%s, %s %d" % (_ra, _dec, star_count))
fig, ax = plt.subplots(figsize=(8,8), dpi=120)
#fig.suptitle(title_string, fontsize=12)
ax.set_xlim(0,360)
ax.set_xlabel(r"$\theta^\circ$")
ax.set_ylabel('Probability')
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1)) 

# i'm not using the actual histogram data atm, but this allows one to 
# inspect the histogram
ax.hist(pm_angles_all, bins=n_bins,density=True,color='black',alpha=0.3)
ax.hist(pm_angles, bins=n_bins,density=True,color='red')

plt.title(title_string)
plt.show()

# 2. show linear regression for the bin

title_string = str.format("PM Correlation for {5} to {6}, {3} to {4}$^\circ$\n r$^2$ = {0:.4} mse = {1:.4} Stars: {2}", 
                          _corr_coeff, _mse, subset_length, angle_start, angle_end,
                         _ra, _dec)

fig, ax = plt.subplots(figsize=(8,8), dpi=120)
#fig.suptitle(title_string, fontsize=12)
plt.title(title_string, fontsize=12)

rms = np.sqrt(np.mean((pmdec_narrow - pmdec_pred) ** 2))
print(rms)
print("RMS error = %.2f" % rms)

ax.set_xlabel('PMRA $mas/yr$')
ax.set_ylabel('PMDec $mas/yr$')

ax.scatter(pmra_narrow, pmdec_narrow, label='Star Subset',s=8,alpha=0.7)
ax.plot(pmra_narrow, pmdec_pred, color='red', label = 'Modeled  PMDec')

ax.plot(pmra_narrow - _mse, pmdec_pred + _mse, ':k', label = '+/- MSE')
ax.plot(pmra_narrow + _mse, pmdec_pred - _mse, ':k')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

#
# 3. plot the RA and Dec for all of the area centers
#

title_string = str.format("Pelican Nebula Subset: {0}/{1} stars", 
                          subset_length, star_count)
fig, ax  = plt.subplots(figsize=(8,8),dpi=120,sharex=True,sharey=True)

plt.title(title_string)

ax.set_xlabel('RA$(hour)$')
ax.set_ylabel('Dec$(deg)$')

ra = stars_pm['ra']
dec = stars_pm['dec']
ax.scatter(ra,dec, label="All Stars",s=2,alpha=0.3)

ra_narr = ra_narrow
dec_narr = dec_narrow
ax.scatter(ra_narr,dec_narr,color='red',label="Subset",s=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.1)
plt.show()

#
# 4. Create a quiver plot of the stars, where the pmra and pmdec determine the 
# orientation of the arrows
#
vel_filename="./eric/PelPM_19h59m08s_34d34m10s_0.05_deg_vel_out.csv"
fig, ax = plt.subplots(figsize=(8,8),dpi=120)

title_string = str.format("Pelican PM distribution: {0}/{1} stars, {2} to {3}$^\circ$", 
                          subset_length, star_count, angle_start, angle_end)
plt.title(title_string)
#fig.suptitle(title_string, fontsize=12)
ax.set_xlabel('RA$(hour)$')
ax.set_ylabel('Dec$(deg)$')
# determine the length of the quiverkey and make the string label
pmra_min = abs(np.min(pmra_narrow))
pmra_max = abs(np.max(pmra_narrow))
pmdec_min = abs(np.min(pmdec_narrow))
pmdec_max = abs(np.max(pmdec_narrow))

pmra_min2 = abs(np.min(pmra))
pmra_max2 = abs(np.max(pmra))
pmdec_min2 = abs(np.min(pmdec))
pmdec_max2 = abs(np.max(pmdec))


pm = [pmra_min, pmra_max, pmdec_min, pmdec_max, pmra_min2, pmra_max2, 
      pmdec_min2, pmdec_max2]
qk_len = int(max(pm) + .5)
qk_str = str(qk_len) + 'mas/yr'

# plot all stars
ax.quiver(ra_narrow, dec_narrow, pmra_narrow, pmdec_narrow, 
          angles='uv',color='red', alpha=1.0, label='Star Subset')

# plot the subset
quiver_plot = ax.quiver(ra, dec, pmra, pmdec, angles='uv',color="black", 
                        alpha = .5, label = 'All Stars')

ax.quiverkey(quiver_plot, X=0.8, Y=.86, U=qk_len, coordinates='figure',label=qk_str, labelpos='E')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.1)
plt.show()

from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# Plot probability histograms for the PM Angles
# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.norm.html
#

n_bins = 72
dens_str = "PDF fit"
stars = read_csv(vel_filename)
angles = []
angles = compute_pm_angles_pandas(stars)

plt_title = "Pelican Nebula (BW Query) "

fig, ax = plt.subplots(sharey=False,figsize=(9,4), dpi=120)
plt.title('Pelican PM: ' + str(len(stars))+ ' stars')
plt.suptitle(position_string)

ax.set_xlabel(r"$\theta^\circ$")
ax.set_ylabel('Probability')
ax.set_xlim(0,360)
ax.grid(False)
ax.axhline(0, color='black', lw=2)
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))  


# get rough parameter estimates 
(mu, sigma ) = norm.fit(angles)

# eyeballed based on the distribution
mu = 210.0
sigma = 15.0
# note that this freezes the histogram. 
n, bins, patches = ax.hist(angles, bins=n_bins, density=True, label='Angle Distribution')

# add a 'best fit' line
y = mlab.normpdf(bins, mu, sigma)

l, = plt.plot(bins, y, 'r--', linewidth=2, label = "PDF")

plt.title(r'$\mathrm{PM\ \theta\: \ \mu=%.3f,\ \sigma=%.3f}$' %(mu, sigma))
plt.legend(loc='best',frameon=False, borderaxespad=0.)
plt.show()

TypeError: compute_pm_angles() takes 1 positional argument but 2 were given